### Jigsaw Pytorch Starter
Model used: Roberta-base
#### RoBERTa
RoBERTa stands for Robustly Optimized Bidirectional Encoder Representations from Transformers.

RoBERTa builds on BERT and modifies key hyperparameters, removing the next-sentence pretraining objective and training with much larger mini-batches and learning rates.

RoBERTa has the same architecture as BERT, but uses a byte-level BPE as a tokenizer (same as GPT-2) and uses a different pretraining scheme.

RoBERTa doesn’t have token_type_ids, you don’t need to indicate which token belongs to which segment. Just separate your segments with the separation token tokenizer.sep_token.

RoBERTa is an extension of BERT with changes to the pretraining procedure. The modifications include:

- Training the model longer, with bigger batches, over more data
- Removing the next sentence prediction objective
- Training on longer sequences
- Dynamically changing the masking pattern applied to the training data. The authors also collect a large new dataset of comparable size to other privately used datasets, to better control for training set size effects.

It was proposed in 2019 by Yinhan Liu Et. al. in RoBERTa: A Robustly Optimized BERT Pretraining Approach

### Introduction
In this article, we will walk through a baseline model for the Jigsaw Rate Severity of Toxic Comments Competition on Kaggle. The goal of the competition is to rank relative ratings of toxicity between comments.

### Install Required Libraries

### Weights & Biases
Machine learning experiment tracking, dataset versioning, and model evaluation
Weights & Biases is the machine learning platform for developers to build better models faster. Use W&B's lightweight, interoperable tools to quickly track experiments, version and iterate on datasets, evaluate model performance, reproduce models, visualize results and spot regressions, and share findings with colleagues.
Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

In [ ]:
!pip install --upgrade wandb  # Machine learning developer platform for building better models, faster with experiment tracking, dataset versioning, and model management

### Import Required Libraries 

In [ ]:
import os  # operating system library
import gc  # Garbage Collector - module provides the ability to disable the collector, tune the collection frequency, and set debugging options
import copy  # The assignment operation does not copy the object, it only creates a reference to the object. 
# For mutable collections, or for collections containing mutable items, a copy is often needed so that it can be modified without changing the original. 
# This module provides general (shallow and deep) copy operations.

import time  # time library
import random  # library for working with random values

import string  # common string operations

# For data manipulation
import pandas as pd  # data analysis library
import numpy as np  # library linear algebra, Fourier transform and random numbers

# Pytorch Imports
import torch  #  a Tensor library like NumPy, with strong GPU support
import torch.nn as nn  # a neural networks library deeply integrated with autograd designed for maximum flexibility
import torch.optim as optim  # Pytorch also has a package with various optimization algorithms.
# We can use the step method from our optimizer to take a forward step, instead of manually updating each parameter.
# This will let us replace our previous manually coded optimization step

from torch.optim import lr_scheduler  # provides several methods to adjust the learning rate based on the number of epochs.
from torch.utils.data import Dataset, DataLoader  # DataLoader and other utility functions for convenience
# DataLoader is responsible for managing batches. You can create a DataLoader from any Dataset. DataLoader makes it easier to iterate over batches. 
# PyTorch has an abstract Dataset class. A Dataset can be anything that has a __len__ function (called by Python’s standard len function) and a __getitem__ function 
# as a way of indexing into it


# Utils
from tqdm import tqdm  # tqdm derives from the Arabic word taqaddum  which can mean "progress," and is an abbreviation for "I love you so much" in Spanish 
# (te quiero demasiado).  this library show a smart progress meter - just wrap any iterable with tqdm(iterable)
from collections import defaultdict  # Usually, a Python dictionary throws a KeyError if you try to get an item with a key that is not currently in the dictionary. 
# The defaultdict in contrast will simply create any items that you try to access (provided of course they do not exist yet). 


# Sklearn Imports
# sklearn - а set of python modules for machine learning and data mining
from sklearn.metrics import mean_squared_error  # Mean squared error regression loss
from sklearn.model_selection import StratifiedKFold, KFold  # Stratified K-Folds cross-validator.
# K-Folds cross-validator. Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default).
# Each fold is then used once as a validation while the k - 1 remaining folds form the training set.

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW  
#  In many cases, the architecture you want to use can be guessed from the name or the path of the 
# pretrained model you are supplying to the from_pretrained() method. AutoClasses are here to do this job for you so that you automatically retrieve the 
# relevant model given the name/path to the pretrained weights/config/vocabulary.
# Instantiating one of AutoConfig, AutoModel, and AutoTokenizer will directly create a class of the relevant architecture.

# For colored terminal text
from colorama import Fore, Back, Style
# ANSI escape character sequences have long been used to produce colored terminal text and cursor positioning on Unix and Macs. 
# Colorama makes this work on Windows, too, by wrapping stdout, stripping ANSI sequences it finds (which would appear as gobbledygook in the output), 
# and converting them into the appropriate win32 calls to modify the state of the terminal. On other platforms, Colorama does nothing.

b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings  # Warning messages are typically issued in situations where it is useful to alert the user of some condition in a program

warnings.filterwarnings("ignore")  # This is the base class of all warning category classes. It is a subclass of Exception. 
# The warnings filter controls whether warnings are ignored, displayed, or turned into errors (raising an exception) 
# "ignore" - never print matching warnings
# For descriptive error messages

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# The beautiful thing of PyTorch's immediate execution model is that you can actually debug your programs. Sometimes, however, the asynchronous nature 
# of CUDA execution makes it hard. Here is a little trick to debug your programs. When you run a PyTorch program using CUDA operations, 
# the program usually doesn't wait until the computation finishes but continues to throw instructions at the GPU until it actually needs 
# a result (e.g. to evaluate using .item() or .cpu() or printing).
# While this behaviour is key to the blazing performance of PyTorch programs, there is a downside: When a cuda operation fails, your program 
# has long gone on to do other stuff. The usual symptom is that you get a very non-descript error at a more or less random place somewhere 
# after the instruction that triggered the error.
# One option in debugging is to move things to CPU. But often, we use libraries or have complex things where that isn't an option. So what now? If we could only get a good traceback, we should find the problem in no time.
# This is how to get a good traceback:You can launch the program with the environment variable CUDA_LAUNCH_BLOCKING set to 1.


from transformers import logging  # lib for logging
logging.set_verbosity_warning()
logging.set_verbosity_error()

### loging in Weights & Biases (W&B) 

In [ ]:
import wandb  # Machine learning experiment tracking, dataset versioning, and model evaluation Weights & Biases is the machine learning platform 
# for developers to build better models faster.


wandb.init(project="my-test-project", entity="doom84")  # Enter your data here (your username and project name)
# When you run this code, you will be prompted to enter API key

# Weights & Biases connection error handler
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \n Get your W&B access token from here: https://wandb.ai/authorize')

### Training Configuration

In [ ]:
def id_generator(size=12, chars=string.ascii_lowercase + string.digits):  # this function Takes random choices from 12 ascii etters and digits
    return ''.join(random.SystemRandom().choice(chars) for _ in range(size))  # returns the resulting 12 character string
 
# In Python, string ascii_lowercase will give the lowercase letters ‘abcdefghijklmnopqrstuvwxyz’.
HASH_NAME = id_generator(size=12)  # will create a test variable and check how our generation function works
print(HASH_NAME)  # display the result

It is always a better idea to have a separate configuration file for your project. In this case, we declare a dictionary named CONFIG, which includes all the required configs.

Fine-tuning Transformer models tend to exhibit training instability. Even with the same hyperparameter values (learning rate, batch size, etc.), distinct random seeds can lead to substantially different results. The issue is even more apparent, especially when using the large variants of Transformers on small datasets.

#### Problem
The instability of the Transformer fine-tuning process has been known since the introduction of BERT, and from then various methods have been proposed to address it.

#### Solution
There are many recently proposed methods to increase few-sample fine-tuning stability and they show a significant performance improvement over simple finetuning methods.
- Debiasing Omission In BertADAM
- Re-Initializing Transformer Layers
- Utilizing Intermediate Layers
- Layer-wise Learning Rate Decay (LLRD)
- Mixout Regularization
- Pre-trained Weight Decay
- Stochastic Weight Averaging


Note 1: These methods are independent and it's not recommended to use all of them at once. Even though mixing two or more techniques might result in improvement but this may not be always true.

In [ ]:
# define the configuration of our model
CONFIG = {"seed": 42, # 2021->42
          "epochs": 3, # 3->5->10->7->3
          "model_name": "roberta-base",
          "train_batch_size": 32,
          "valid_batch_size": 64,
          "max_length": 128,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 25, #5->10->5->10
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "hash_name": HASH_NAME
          }

# 10folds * 3epochs * 13 minuts = 390 minuts = 6,5 hours  on GPU kaggle
# if Using GPU: Tesla K80 google colab it will turn out many times faster = 1,5 часа

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])
CONFIG['group'] = f'{HASH_NAME}-Baseline'

### Set Seed for Reproducibility

In [ ]:
# Sets the seed of the entire notebook so results are the same every time we run.
# This is for REPRODUCIBILITY
def set_seed(seed=42): 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

### Read the Data 

In [ ]:
df = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")  # create a variable dataframe containing data from the original competition data file  
print(df.shape)  # display statistics for in this file
df.head()  # display the first 5 rows of the dataframe table

### Create Folds

In [ ]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])  # set the parameters for splitting our dataframe into data for training and testing

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df.worker)):  # dataframe splitting
    df.loc[val_ , "kfold"] = int(fold)
    
df["kfold"] = df["kfold"].astype(int)  # add one more column of folder number to the original dataframe
df.head()  # display the first 5 rows of the dataframe table

### Dataset Class

In [ ]:
class JigsawDataset(Dataset):  # create a JigsawDataset class
    def __init__(self, df, tokenizer, max_length):  # initialization of the class at the input of the dataframe, tokenizer, max_length
        # set the class attributes
        # the __init__ function is run once when instantiating the Dataset object.
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.more_toxic = df['more_toxic'].values
        self.less_toxic = df['less_toxic'].values
        
    def __len__(self):  # the __len__ function returns the number of samples in our dataset.
        return len(self.df)
    
    def __getitem__(self, index):  # __getitem__ function loads and returns a sample from the dataset at the given index index. Based on the index, 
        # it identifies the data location on disk, encodes the data, and sets the target = 1. Finally it returns tensors of the ids, mask, target. 
        # Remember that we are using the roberta model here which does not have token_type_ids. 
        
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        # Since no Target variable is present in the dataset (remember we explicitly specified Target = 1 in the dataset class), we use the Margin Ranking Loss
        # It creates a criterion that measures the loss given inputs x1, x2, two 1D mini-batch Tensors, and a label 1D mini-batch tensor y (containing 1 or -1).
        # If y = 1, then it assumed the first input should be ranked higher (should have a larger value) than the second input, and vice-versa for y = -1.
        # For the same reason, we set Target = 1 earlier.
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']
        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        
        return {  # returns the obtained values
            'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
            'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
            'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
            'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long)
        }

### Create Model

In [ ]:
class JigsawModel(nn.Module):  # create a JigsawModel class
    def __init__(self, model_name):  # initialization of the class at the input of the dataframe, tokenizer, max_length
        # set the class attributes
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        
        return outputs  # returns the obtained values

### Loss Function

![](https://i.imgur.com/qYwVt8V.jpg)


In [ ]:
def criterion(outputs1, outputs2, targets):  # Creates a criterion that measures the loss
    return nn.MarginRankingLoss(margin=CONFIG['margin'])(outputs1, outputs2, targets)

### Training Function 

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):  # one epoch training function
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    
    for step, data in bar:
        more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.long)
        more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.long)
        less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.long)
        less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = more_toxic_ids.size(0)

        more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
        less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
        
        loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss  # returns the result of the training function for one epoch

### Validation Function

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):  # one epoch check function
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.long)
        more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.long)
        less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.long)
        less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = more_toxic_ids.size(0)

        more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
        less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
        
        loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss  # returns the result of the check function for one epoch

### Run Training

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):  # general training function
    # To automatically log gradients
    wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
        # Log the metrics
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"Loss-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
def prepare_loaders(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = JigsawDataset(df_train, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
    valid_dataset = JigsawDataset(df_valid, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

### Start Training 
With the data, model and hardware prepared we are now ready to train the model. We use N Fold Training as a Cross-Validation Strategy here.

In [ ]:
for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    run = wandb.init(project='Jigsaw', 
                     config=CONFIG,
                     job_type='Train',
                     group=CONFIG['group'],
                     tags=['roberta-base', f'{HASH_NAME}', 'margin-loss'],
                     name=f'{HASH_NAME}-fold-{fold}',
                     anonymous='must')
    
    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(fold=fold)
    
    model = JigsawModel(CONFIG['model_name'])
    model.to(CONFIG['device'])
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold)
    
    run.finish()
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()
print('Сompleted training and saving models!')